In [2]:
%load_ext autoreload
%autoreload 2
import logging
from pathlib import Path
from typing import Dict, List, Optional, Tuple, Union

import duckdb
import pandas as pd
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

from planter.database.schema.schema_version import (SCHEMA_VERSIONS,
                                                    ensure_compatibility,
                                                    get_db_schema_version)


In [1]:
from planter.database.utils.duckdb_utils import (
    extract_representative_sequences, 
    merge_duckdbs, 
    update_duckdb_with_cluster_info,
    create_duckdb
)


In [41]:
samples = ['SRR12068547', 'SRR12068548', 'SRR12068549', 'SRR12068550', 'SRR12068551', 'SRR12068552']
outdir = Path('/mnt/data4/recombia.planter')

for sample in samples:
    create_duckdb(
        sample_id=sample,
        outdir=outdir,
        duckdb_out=outdir / f'{sample}/{sample}.duckdb'
    )

In [42]:
import duckdb
import pandas as pd
from pathlib import Path

def validate_duckdb_schema(db_path):
    """Validate schema and relationships in the database."""
    print(f"Validating database: {db_path}")
    
    with duckdb.connect(db_path) as con:
        # 1. Get list of all tables
        print("\n=== Tables in database ===")
        tables = con.execute("SELECT name FROM sqlite_master WHERE type='table'").fetchall()
        for table in tables:
            table_name = table[0]
            count = con.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
            print(f"Table: {table_name} - {count} rows")
        
        # 2. Check schema of each table
        print("\n=== Schema for each table ===")
        for table in tables:
            table_name = table[0]
            print(f"\nSchema for {table_name}:")
            schema = con.execute(f"PRAGMA table_info({table_name})").fetchall()
            for col in schema:
                print(f"  {col[1]} ({col[2]}){' PRIMARY KEY' if col[5] > 0 else ''}")
        
        # 3. Check the gene-protein relationships
        print("\n=== Gene-Protein Relationships ===")
        try:
            gene_protein_stats = con.execute("""
                SELECT 
                    COUNT(DISTINCT gene_seqhash_id) AS total_genes,
                    COUNT(DISTINCT protein_seqhash_id) AS total_proteins,
                    COUNT(*) AS total_relationships
                FROM gene_protein_map
            """).fetchone()
            
            print(f"Total genes: {gene_protein_stats[0]}")
            print(f"Total proteins: {gene_protein_stats[1]}")
            print(f"Total gene-protein relationships: {gene_protein_stats[2]}")
            
            # Check for genes with multiple proteins
            multi_protein_genes = con.execute("""
                SELECT gene_seqhash_id, COUNT(protein_seqhash_id) as protein_count
                FROM gene_protein_map
                GROUP BY gene_seqhash_id
                HAVING COUNT(protein_seqhash_id) > 1
                ORDER BY COUNT(protein_seqhash_id) DESC
                LIMIT 5
            """).fetchall()
            
            if multi_protein_genes:
                print("\nTop 5 genes with multiple proteins:")
                for gene in multi_protein_genes:
                    print(f"  Gene {gene[0]} has {gene[1]} proteins")
                    # Sample of proteins for this gene
                    proteins = con.execute(f"""
                        SELECT protein_seqhash_id 
                        FROM gene_protein_map 
                        WHERE gene_seqhash_id = '{gene[0]}'
                        LIMIT 3
                    """).fetchall()
                    for protein in proteins:
                        print(f"    - {protein[0]}")
        except Exception as e:
            print(f"Error checking gene-protein relationships: {e}")
        
        # 4. Check expression data and linkage to genes
        print("\n=== Expression Data ===")
        try:
            expr_stats = con.execute("""
                SELECT 
                    COUNT(DISTINCT gene_seqhash_id) AS genes_with_expression,
                    COUNT(DISTINCT sample_id) AS samples_with_expression,
                    AVG(tpm) AS avg_tpm
                FROM expression
            """).fetchone()
            
            print(f"Genes with expression data: {expr_stats[0]}")
            print(f"Samples with expression data: {expr_stats[1]}")
            print(f"Average TPM: {expr_stats[2]:.2f}")
            
            # Check gene-expression-protein linkage
            gene_expr_protein = con.execute("""
                SELECT 
                    COUNT(DISTINCT e.gene_seqhash_id) AS genes_with_expr_and_protein,
                    COUNT(DISTINCT gpm.protein_seqhash_id) AS proteins_linked_to_expr
                FROM expression e
                JOIN gene_protein_map gpm ON e.gene_seqhash_id = gpm.gene_seqhash_id
            """).fetchone()
            
            print(f"Genes with both expression and protein mappings: {gene_expr_protein[0]}")
            print(f"Proteins linked to genes with expression: {gene_expr_protein[1]}")
        except Exception as e:
            print(f"Error checking expression data: {e}")
        
        # 5. Check annotations
        print("\n=== Annotation Data ===")
        try:
            anno_stats = con.execute("""
                SELECT 
                    COUNT(*) AS total_annotations,
                    COUNT(DISTINCT sample_id) AS samples_with_annotations
                FROM annotations
            """).fetchone()
            
            print(f"Total annotated sequences: {anno_stats[0]}")
            print(f"Samples with annotations: {anno_stats[1]}")
            
            # Check annotation-sequence-gene-expression linkage
            anno_gene_expr = con.execute("""
                SELECT 
                    COUNT(DISTINCT a.seqhash_id) AS annotated_proteins,
                    COUNT(DISTINCT gpm.gene_seqhash_id) AS genes_of_annotated_proteins,
                    COUNT(DISTINCT e.gene_seqhash_id) AS genes_with_annotation_and_expression
                FROM annotations a
                JOIN sequences s ON a.seqhash_id = s.seqhash_id
                JOIN gene_protein_map gpm ON s.seqhash_id = gpm.protein_seqhash_id
                LEFT JOIN expression e ON gpm.gene_seqhash_id = e.gene_seqhash_id
            """).fetchone()
            
            print(f"Annotated proteins: {anno_gene_expr[0]}")
            print(f"Genes of annotated proteins: {anno_gene_expr[1]}")
            print(f"Genes with both annotation and expression: {anno_gene_expr[2]}")
        except Exception as e:
            print(f"Error checking annotation data: {e}")
        
        # 6. Check clusters
        print("\n=== Cluster Data ===")
        try:
            cluster_stats = con.execute("""
                SELECT 
                    COUNT(DISTINCT cluster_id) AS total_clusters,
                    AVG(size) AS avg_cluster_size,
                    MAX(size) AS largest_cluster
                FROM clusters
            """).fetchone()
            
            print(f"Total clusters: {cluster_stats[0]}")
            print(f"Average cluster size: {cluster_stats[1]:.2f}")
            print(f"Largest cluster size: {cluster_stats[2]}")
            
            # Check cluster members
            member_stats = con.execute("""
                SELECT 
                    COUNT(DISTINCT seqhash_id) AS unique_members,
                    COUNT(*) AS total_membership_records
                FROM cluster_members
            """).fetchone()
            
            print(f"Unique sequences in clusters: {member_stats[0]}")
            print(f"Total cluster membership records: {member_stats[1]}")
            
            # Get top clusters
            top_clusters = con.execute("""
                SELECT cluster_id, size
                FROM clusters
                ORDER BY size DESC
                LIMIT 3
            """).fetchall()
            
            if top_clusters:
                print("\nTop 3 largest clusters:")
                for cluster in top_clusters:
                    print(f"  Cluster {cluster[0]}: {cluster[1]} members")
        except Exception as e:
            print(f"Error checking cluster data: {e}")

# Usage
db_path = "/mnt/data4/recombia.planter/SRR12068547/SRR12068547.duckdb"
validate_duckdb_schema(db_path)

Validating database: /mnt/data4/recombia.planter/SRR12068547/SRR12068547.duckdb

=== Tables in database ===
Table: annotations - 251 rows
Table: clusters - 0 rows
Table: cluster_members - 0 rows
Table: ec_numbers - 114 rows
Table: expression - 365 rows
Table: gene_protein_map - 398 rows
Table: go_terms - 267 rows
Table: kegg_info - 236 rows
Table: schema_version - 0 rows
Table: sequences - 398 rows
Table: sra_metadata - 1 rows

=== Schema for each table ===

Schema for annotations:
  seqhash_id (VARCHAR) PRIMARY KEY
  seed_ortholog (VARCHAR)
  evalue (DOUBLE)
  score (DOUBLE)
  eggnog_ogs (VARCHAR)
  max_annot_lvl (VARCHAR)
  cog_category (VARCHAR)
  description (VARCHAR)
  preferred_name (VARCHAR)
  sample_id (VARCHAR)

Schema for clusters:
  cluster_id (VARCHAR) PRIMARY KEY
  representative_seqhash_id (VARCHAR)
  size (INTEGER)

Schema for cluster_members:
  seqhash_id (VARCHAR) PRIMARY KEY
  cluster_id (VARCHAR) PRIMARY KEY

Schema for ec_numbers:
  seqhash_id (VARCHAR) PRIMARY KEY


In [43]:
merge_duckdbs(
    duckdb_paths=[outdir / f'{sample}/{sample}.duckdb' for sample in samples],
    master_db_path=outdir / 'master.duckdb',
    schema_sql_path=Path('../planter/database/schema/migrations/004_add_gene_protein_map.sql'),
    upgrade_schema=True,
    target_schema_version=None
)

'/mnt/data4/recombia.planter/master.duckdb'

In [44]:
def check_tables_in_db(db_path):
    """
    Check which tables exist in a DuckDB database.
    
    Args:
        db_path: Path to the DuckDB file
        
    Returns:
        Dictionary of table names and their row counts
    """
    db_path = Path(db_path)
    
    if not db_path.exists():
        print(f"Database file does not exist: {db_path}")
        return {}
    
    try:
        with duckdb.connect(str(db_path)) as conn:
            # Get all table names
            tables = conn.execute(
                "SELECT name FROM sqlite_master WHERE type='table'"
            ).fetchall()
            
            # Create a dictionary to store table info
            table_info = {}
            
            # For each table, get row count and column info
            for table_row in tables:
                table_name = table_row[0]
                
                # Get row count
                try:
                    row_count = conn.execute(f"SELECT COUNT(*) FROM {table_name}").fetchone()[0]
                except Exception as e:
                    row_count = f"Error: {str(e)}"
                
                # Get column info
                try:
                    columns = conn.execute(f"PRAGMA table_info({table_name})").fetchall()
                    column_names = [col[1] for col in columns]
                except Exception as e:
                    column_names = [f"Error: {str(e)}"]
                
                # Store table info
                table_info[table_name] = {
                    "row_count": row_count,
                    "columns": column_names
                }
            
            return table_info
    except Exception as e:
        print(f"Error connecting to database: {str(e)}")
        return {}
    
# db_path = "/mnt/data4/recombia.planter/SRR12068547/SRR12068547.duckdb"
db_path = '/mnt/data4/recombia.planter/master.duckdb'
check_tables_in_db(db_path)

{'annotations': {'row_count': 2049722,
  'columns': ['seqhash_id',
   'seed_ortholog',
   'evalue',
   'score',
   'eggnog_ogs',
   'max_annot_lvl',
   'cog_category',
   'description',
   'preferred_name',
   'sample_id']},
 'clusters': {'row_count': 433785,
  'columns': ['cluster_id', 'representative_seqhash_id', 'size']},
 'cluster_members': {'row_count': 2489980,
  'columns': ['seqhash_id', 'cluster_id']},
 'ec_numbers': {'row_count': 571387, 'columns': ['seqhash_id', 'ec_number']},
 'expression': {'row_count': 1586,
  'columns': ['gene_seqhash_id',
   'sample_id',
   'tpm',
   'num_reads',
   'effective_length']},
 'expression_backup': {'row_count': 3091553,
  'columns': ['gene_seqhash_id',
   'sample_id',
   'tpm',
   'num_reads',
   'effective_length']},
 'gene_protein_map': {'row_count': 745417,
  'columns': ['gene_seqhash_id', 'protein_seqhash_id']},
 'go_terms': {'row_count': 74845016, 'columns': ['seqhash_id', 'go_term']},
 'kegg_info': {'row_count': 1626,
  'columns': ['seq

In [66]:
cluster_path = '/mnt/data4/planter_outputs/tmp/newClusterDB.tsv'
update_duckdb_with_cluster_info(
    db_path=db_path,
    tsv_path=cluster_path,
    upgrade_schema=True
)


Error updating cluster information: Constraint Error: Violates foreign key constraint because key "cluster_id: v1_DLS_19696ccb77b302cd0cdff9203ea61ab6436f11d2972fe64be97b00a5961373da.p1" is still referenced by a foreign key in a different table


ConstraintException: Constraint Error: Violates foreign key constraint because key "cluster_id: v1_DLS_19696ccb77b302cd0cdff9203ea61ab6436f11d2972fe64be97b00a5961373da.p1" is still referenced by a foreign key in a different table

In [63]:
import pandas as pd
clusters = pd.read_csv(cluster_path, sep='\t', header=None, names=['cluster_id', 'seqhash_id'])
clusters[clusters['cluster_id'] == 'v1_DLS_42f03501d692b370647a4bac7059434916aa1d21968fca917aa214f32f2ced17.p1']









,cluster_id,seqhash_id
41418,v1_DLS_42f03501d692b370647a4bac7059434916aa1d2...,v1_DLS_42f03501d692b370647a4bac7059434916aa1d2...
41419,v1_DLS_42f03501d692b370647a4bac7059434916aa1d2...,v1_DLS_1d0a4dd05e3a9578522ecb4ae72d94bc9336371...
41420,v1_DLS_42f03501d692b370647a4bac7059434916aa1d2...,v1_DLS_6daf155fe0272972a572f597efd9345da48f108...
41421,v1_DLS_42f03501d692b370647a4bac7059434916aa1d2...,v1_DLS_7955ebc49cf7dcab1836f5b50e0fdba5f6db33f...
41422,v1_DLS_42f03501d692b370647a4bac7059434916aa1d2...,v1_DLS_5a617794888abd2a45f0b5c8e408df74aa0468e...
41423,v1_DLS_42f03501d692b370647a4bac7059434916aa1d2...,v1_DLS_f6857abe786b2565220f2a06a7f4fbca3924777...
41424,v1_DLS_42f03501d692b370647a4bac7059434916aa1d2...,v1_DLS_1cb17d74a3374f2468d7077af03aeb9eca566e0...
41425,v1_DLS_42f03501d692b370647a4bac7059434916aa1d2...,v1_DLS_8bd89956ed7895572c33cb9d6f67af2c37f46c7...
41426,v1_DLS_42f03501d692b370647a4bac7059434916aa1d2...,v1_DLS_1caf6982d5ee3ae3e7ca16db2119f88676e9b7a...
41427,v1_DLS_42f03501d692b370647a4bac7059434916aa1d2...,v1_DLS_a4018c7a2e2099089391ec0c1d72a2a68d72588...
